In [24]:
import pandas as pd
import numpy as np
import warnings
import scipy as sp
from datetime import datetime
from IPython.display import display
import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from IPython.core.display import display, HTML
from plotly.subplots import make_subplots
from IPython.display import display
autoscroll_set_on_load = True
autoscroll_starting_threshold = -1
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df = pd.read_excel('data/LIGHT4DEAF_Neurocognition_Avril_2020_1_FULL_ADULTES_20200504.xlsx')

In [25]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [27]:
df = df[df["Investigateur"].notnull()]

In [28]:
cardinal = len(df)

In [29]:
now = datetime.now()
df['Date de naissance du patient'] = pd.to_datetime(df['Date de naissance du patient'], format='%m/%d/%y')
df['Date de naissance du patient'] = df['Date de naissance du patient'].where(df['Date de naissance du patient'] < now, df['Date de naissance du patient'] -  np.timedelta64(100, 'Y')) 
df['age'] = (now - df['Date de naissance du patient']).astype('<m8[Y]')

In [30]:
#Patients adultes : type
df_type1 = df[df["Type"]=='Type 1']
df_type2 = df[df["Type"]=='Type 2']
df_type3 = df[df["Type"]=='Type 3']
df_type_nan = df[df["Type"].isna()]

In [31]:
display(HTML('<h1>L i g h t 4 d e a f</h1>'))
display(HTML('<h2>Neurocognition - TAP</h2>'))
display(HTML('<p>Effectif total (TAP : Oui) : '+str(len(df[(df['TAP']=='Oui')]))+'</hp>'))

In [32]:
cols = df.columns.values.tolist()
cols.index("TAP")
cols.index('Autre commentaire TAP')
arr = cols[cols.index('TAP'):cols.index('Autre commentaire TAP')]
arr.insert(0, "ID_SECONDAIRE")
arr.insert(1, "age")
arr.insert(2, "Type")
arr.insert(3, "Correction visuelle")
arr.insert(4, "Prothèse auditive DROITE")
arr.insert(5, "Prothèse auditive GAUCHE")
arr.insert(6, "Implant cochléaire DROIT")
arr.insert(7, "Implant cochléaire GAUCHE")
arr.insert(8, "Interprète LSF")
arr.insert(9, "Nombre d'années d'études")

In [33]:
df = df.loc[:,arr]

In [34]:
df.loc[df['Type']=="Type 2", 'Type_color']="#B3D9FF"
df.loc[df['Type']=="Type 1", 'Type_color']="#33ffc9"
df.loc[df['Type']=="Type 3", 'Type_color']="#FFC433"
df.loc[df['Type'].isna(), 'Type_color']="#C5C5C5"
df['Type'].fillna('Type inconnu', inplace= True)

In [55]:
display(HTML('<h4>1. Alerte phasique </h4>'))

df_pop = df[(df['TAP']=='Oui') & (df['Alerte phasique avec avertisseur : médiane - Temps (ms)'].notna())]
total = len(df_pop)
age_avg = round(df_pop["age"].mean())
age_min = df_pop["age"].min()
age_max = df_pop["age"].max()

#Patients ayant une aide auditive
df_coaud = df_pop[(df_pop["Prothèse auditive DROITE"]!="Aucune") | (df_pop["Prothèse auditive GAUCHE"]!="Aucune") | (df_pop["Implant cochléaire DROIT"]!="Aucun") | (df_pop["Implant cochléaire GAUCHE"]!="Aucun")]
#Patients ayant uniquement une aide auditive
df_coaud_u = df_coaud[df_coaud["Correction visuelle"]=="Aucune"]
coaud_u = len(df_coaud_u)

#Patients ayant une correction visuelle
df_covis = df_pop[df_pop["Correction visuelle"]!="Aucune"]
#Patients ayant uniquement une correction visuelle
df_covis_u = df_covis[((df_covis["Prothèse auditive DROITE"]=="Aucune") & (df_covis["Prothèse auditive GAUCHE"]=="Aucune") & (df_covis["Implant cochléaire DROIT"]=="Aucun") & (df_covis["Implant cochléaire GAUCHE"]=="Aucun"))]
covis_u = len(df_covis_u)

#Patients ayant une aide auditive + visuelle
df_coaud_covis = df_coaud[df_coaud["Correction visuelle"]!="Aucune"]
coaud_covis = len(df_coaud_covis)

intlsf = len(df_pop[df_pop["Interprète LSF"]=="Oui"])

columns= ['','Valeur']
myarr = [["Effectif",
            'Moyenne d’âge',
            'Âge minimum',
            'Âge maximum',
            'Patients ayant uniquement une correction visuelle',
            'Patients ayant uniquement une correction auditive',
            'Patients ayant une correction auditive + une correction visuelle',
            'Nombre de personnes ayant recours un interprète']]
for i in columns[1:]:
    myarr.append([total, age_avg, age_min, age_max, covis_u, coaud_u, coaud_covis, intlsf])

str_data =  go.Table(
                columnwidth = [30,5],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=str_data)
fig.show()

###
display(HTML("<p>Répartition du nombre d'années d'études :</p>"))
ranges = [0,5,10,12,15,20]

df_study = df_pop["Nombre d'années d'études"].copy()

df_study = df_study.groupby(pd.cut(df_pop["Nombre d'années d'études"], ranges, include_lowest=True)).count()
df_study = pd.DataFrame(df_study)
df_study = df_study.rename(columns={"Nombre d'années d'études":'Effectif'})
year_study = ["0 à 5", "5 à 10", "10 à 12", "12 à 15", "15 à 20"]
df_study["Nombre d'années d'études"] = year_study
df_study = df_study[["Nombre d'années d'études", "Effectif"]]
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_study.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_study["Effectif"], df_study["Nombre d'années d'études"]],
               fill_color='lavender',
               align='left'))
])

fig.show()

fig = px.bar(df_study, x="Nombre d'années d'études", y="Effectif", color="Effectif")
fig.update_layout(showlegend=False)
fig.show()

myheight = 350
mywidth=1300
columns= ['Calcul', 'Alerte phasique avec avertisseur : médiane - Temps (ms)','Alerte phasique avec avertisseur : médiane - Note T','Alerte phasique avec avertisseur : médiane - Rang percentile']
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()

columns= ['Calcul','Alerte phasique avec avertisseur : médiane - Nb erreurs','Alerte phasique avec avertisseur - Nb omissions',
         'Alerte phasique sans avertisseur : médiane  - Temps (ms)',
         'Alerte phasique avec avertisseur : médiane - Temps (ms)']
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()
##
columns= ['Calcul',
          'Alerte phasique sans avertisseur : médiane  - Temps (ms)',
        'Alerte phasique sans avertisseur : médiane  -  Note T',
         'Alerte phasique sans avertisseur : médiane  -  Rang percentile']
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()
###
columns= ['Calcul','Alerte phasique sans avertisseur : médiane  - Nb erreurs',
         'Alerte phasique sans avertisseur : médiane  -  Nb omissions'
         ]
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()
####
columns= ['Calcul','Index - Temps (ms)','Index - Note T','Index - Rang percentile']
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()

In [51]:
display(HTML('<h4>2. Attention soutenue </h4>'))

df_pop = df[(df['TAP']=='Oui') & (df['Attention soutenue : médiane - Note brute'].notna())]
total = len(df_pop)
age_avg = round(df_pop["age"].mean())
age_min = df_pop["age"].min()
age_max = df_pop["age"].max()

#Patients ayant une aide auditive
df_coaud = df_pop[(df_pop["Prothèse auditive DROITE"]!="Aucune") | (df_pop["Prothèse auditive GAUCHE"]!="Aucune") | (df_pop["Implant cochléaire DROIT"]!="Aucun") | (df_pop["Implant cochléaire GAUCHE"]!="Aucun")]
#Patients ayant uniquement une aide auditive
df_coaud_u = df_coaud[df_coaud["Correction visuelle"]=="Aucune"]
coaud_u = len(df_coaud_u)

#Patients ayant une correction visuelle
df_covis = df_pop[df_pop["Correction visuelle"]!="Aucune"]
#Patients ayant uniquement une correction visuelle
df_covis_u = df_covis[((df_covis["Prothèse auditive DROITE"]=="Aucune") & (df_covis["Prothèse auditive GAUCHE"]=="Aucune") & (df_covis["Implant cochléaire DROIT"]=="Aucun") & (df_covis["Implant cochléaire GAUCHE"]=="Aucun"))]
covis_u = len(df_covis_u)

#Patients ayant une aide auditive + visuelle
df_coaud_covis = df_coaud[df_coaud["Correction visuelle"]!="Aucune"]
coaud_covis = len(df_coaud_covis)

intlsf = len(df_pop[df_pop["Interprète LSF"]=="Oui"])

columns= ['','Valeur']
myarr = [["Effectif",
            'Moyenne d’âge',
            'Âge minimum',
            'Âge maximum',
            'Patients ayant uniquement une correction visuelle',
            'Patients ayant uniquement une correction auditive',
            'Patients ayant une correction auditive + une correction visuelle',
            'Nombre de personnes ayant recours un interprète']]
for i in columns[1:]:
    myarr.append([total, age_avg, age_min, age_max, covis_u, coaud_u, coaud_covis, intlsf])

str_data =  go.Table(
                columnwidth = [30,5],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=str_data)
fig.show()

###
display(HTML("<p>Répartition du nombre d'années d'études :</p>"))
ranges = [0,5,10,12,15,20]

df_study = df_pop["Nombre d'années d'études"].copy()

df_study = df_study.groupby(pd.cut(df_pop["Nombre d'années d'études"], ranges, include_lowest=True)).count()
df_study = pd.DataFrame(df_study)
df_study = df_study.rename(columns={"Nombre d'années d'études":'Effectif'})
year_study = ["0 à 5", "5 à 10", "10 à 12", "12 à 15", "15 à 20"]
df_study["Nombre d'années d'études"] = year_study
df_study = df_study[["Nombre d'années d'études", "Effectif"]]
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_study.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_study["Effectif"], df_study["Nombre d'années d'études"]],
               fill_color='lavender',
               align='left'))
])

fig.show()

fig = px.bar(df_study, x="Nombre d'années d'études", y="Effectif", color="Effectif")
fig.update_layout(showlegend=False)
fig.show()
#
myheight = 350
mywidth=1300
columns= ['Calcul','Attention soutenue : médiane - Note brute',
          'Attention soutenue : médiane - Note T','Attention soutenue : médiane - Rang percentile']
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()
##
columns= ['Calcul','Att soutenue : Nb omissions - Note brute',
          'Att soutenue : Nb omissions - Note T',
          'Att soutenue : Nb omissions - Rang percentile'
             ]
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=1000, height=myheight)
fig.show()
###
columns= ['Calcul','Att soutenue : Nb erreurs - Note brute',
          'Att soutenue : Nb erreurs - Note T',
          'Att soutenue : Nb erreurs - Rang percentile'
             ]
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()


In [56]:
display(HTML('<h4>3. MDT </h4>'))

df_pop = df[(df['TAP']=='Oui') & (df['MDT Condition 3 : médiane - Note brute'].notna())]
total = len(df_pop)
age_avg = round(df_pop["age"].mean())
age_min = df_pop["age"].min()
age_max = df_pop["age"].max()

#Patients ayant une aide auditive
df_coaud = df_pop[(df_pop["Prothèse auditive DROITE"]!="Aucune") | (df_pop["Prothèse auditive GAUCHE"]!="Aucune") | (df_pop["Implant cochléaire DROIT"]!="Aucun") | (df_pop["Implant cochléaire GAUCHE"]!="Aucun")]
#Patients ayant uniquement une aide auditive
df_coaud_u = df_coaud[df_coaud["Correction visuelle"]=="Aucune"]
coaud_u = len(df_coaud_u)

#Patients ayant une correction visuelle
df_covis = df_pop[df_pop["Correction visuelle"]!="Aucune"]
#Patients ayant uniquement une correction visuelle
df_covis_u = df_covis[((df_covis["Prothèse auditive DROITE"]=="Aucune") & (df_covis["Prothèse auditive GAUCHE"]=="Aucune") & (df_covis["Implant cochléaire DROIT"]=="Aucun") & (df_covis["Implant cochléaire GAUCHE"]=="Aucun"))]
covis_u = len(df_covis_u)

#Patients ayant une aide auditive + visuelle
df_coaud_covis = df_coaud[df_coaud["Correction visuelle"]!="Aucune"]
coaud_covis = len(df_coaud_covis)

intlsf = len(df_pop[df_pop["Interprète LSF"]=="Oui"])

columns = ['','Valeur']
myarr = [["Effectif",
            'Moyenne d’âge',
            'Âge minimum',
            'Âge maximum',
            'Patients ayant uniquement une correction visuelle',
            'Patients ayant uniquement une correction auditive',
            'Patients ayant une correction auditive + une correction visuelle',
            'Nombre de personnes ayant recours à un interprète']]
for i in columns[1:]:
    myarr.append([total, age_avg, age_min, age_max, covis_u, coaud_u, coaud_covis, intlsf])

str_data =  go.Table(
                columnwidth = [30,5],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=str_data)
fig.show()

###
display(HTML("<p>Répartition du nombre d'années d'études :</p>"))
ranges = [0,5,10,12,15,20]

df_study = df_pop["Nombre d'années d'études"].copy()

df_study = df_study.groupby(pd.cut(df_pop["Nombre d'années d'études"], ranges, include_lowest=True)).count()
df_study = pd.DataFrame(df_study)
df_study = df_study.rename(columns={"Nombre d'années d'études":'Effectif'})
year_study = ["0 à 5", "5 à 10", "10 à 12", "12 à 15", "15 à 20"]
df_study["Nombre d'années d'études"] = year_study
df_study = df_study[["Nombre d'années d'études", "Effectif"]]
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_study.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_study["Effectif"], df_study["Nombre d'années d'études"]],
               fill_color='lavender',
               align='left'))
])

fig.show()

fig = px.bar(df_study, x="Nombre d'années d'études", y="Effectif", color="Effectif")
fig.update_layout(showlegend=False)
fig.show()

myheight=350
mywidth=1300
columns= ['Calcul','MDT Condition 3 : médiane - Note brute', 'MDT Condition 3 : médiane - Note T','MDT Condition 3 : médiane - Rang percentile']
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()
##
columns= ['Calcul','MDT cond 3 : Nb erreurs - Note brute',
          'MDT cond 3 : Nb erreurs - Note T',
          'MDT cond 3 : Nb erreurs - Rang percentile'
         ]
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()

In [58]:
display(HTML('<h4>4. Go / No go </h4>'))

df_pop = df[(df['TAP']=='Oui') & (df['Go / No go Condition 1 : médiane - Note brute'].notna())]
total = len(df_pop)
age_avg = round(df_pop["age"].mean())
age_min = df_pop["age"].min()
age_max = df_pop["age"].max()

#Patients ayant une aide auditive
df_coaud = df_pop[(df_pop["Prothèse auditive DROITE"]!="Aucune") | (df_pop["Prothèse auditive GAUCHE"]!="Aucune") | (df_pop["Implant cochléaire DROIT"]!="Aucun") | (df_pop["Implant cochléaire GAUCHE"]!="Aucun")]
#Patients ayant uniquement une aide auditive
df_coaud_u = df_coaud[df_coaud["Correction visuelle"]=="Aucune"]
coaud_u = len(df_coaud_u)

#Patients ayant une correction visuelle
df_covis = df_pop[df_pop["Correction visuelle"]!="Aucune"]
#Patients ayant uniquement une correction visuelle
df_covis_u = df_covis[((df_covis["Prothèse auditive DROITE"]=="Aucune") & (df_covis["Prothèse auditive GAUCHE"]=="Aucune") & (df_covis["Implant cochléaire DROIT"]=="Aucun") & (df_covis["Implant cochléaire GAUCHE"]=="Aucun"))]
covis_u = len(df_covis_u)

#Patients ayant une aide auditive + visuelle
df_coaud_covis = df_coaud[df_coaud["Correction visuelle"]!="Aucune"]
coaud_covis = len(df_coaud_covis)

intlsf = len(df_pop[df_pop["Interprète LSF"]=="Oui"])

columns= ['','Valeur']
myarr = [["Effectif",
            'Moyenne d’âge',
            'Âge minimum',
            'Âge maximum',
            'Patients ayant uniquement une correction visuelle',
            'Patients ayant uniquement une correction auditive',
            'Patients ayant une correction auditive + une correction visuelle',
            'Nombre de personnes ayant recours un interprète']]
for i in columns[1:]:
    myarr.append([total, age_avg, age_min, age_max, covis_u, coaud_u, coaud_covis, intlsf])

str_data =  go.Table(
                columnwidth = [30,5],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=str_data)
fig.show()

###
display(HTML("<p>Répartition du nombre d'années d'études :</p>"))
ranges = [0,5,10,12,15,20]

df_study = df_pop["Nombre d'années d'études"].copy()

df_study = df_study.groupby(pd.cut(df_pop["Nombre d'années d'études"], ranges, include_lowest=True)).count()
df_study = pd.DataFrame(df_study)
df_study = df_study.rename(columns={"Nombre d'années d'études":'Effectif'})
year_study = ["0 à 5", "5 à 10", "10 à 12", "12 à 15", "15 à 20"]
df_study["Nombre d'années d'études"] = year_study
df_study = df_study[["Nombre d'années d'études", "Effectif"]]
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_study.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_study["Effectif"], df_study["Nombre d'années d'études"]],
               fill_color='lavender',
               align='left'))
])

fig.show()

fig = px.bar(df_study, x="Nombre d'années d'études", y="Effectif", color="Effectif")
fig.update_layout(showlegend=False)
fig.show()
myheight=350
mywidth=1300
#
columns= ['Calcul',
          'Go / No go Condition 1 : médiane - Note brute',
          'Go / No go Condition 1 : médiane - Note T',
          'Go / No go Condition 1 : médiane - Rang percentile']
myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()
###
columns= ['Calcul',
        'GNG cond 1 : Nb omissions - Note brute',
        'GNG cond 1 : Nb omissions - Note T',
        'GNG cond 1 : Nb omissions - Rang percentile']

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()
####
columns= ['Calcul',
        'GNG cond 1 : Nb erreurs - Note brute',
         'GNG cond 1 : Nb erreurs - Note T',
         'GNG cond 1 : Nb erreurs - Rang percentile']

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Max.', 'Min.']]

for i in columns[1:]:
    effectif = len(df_pop[df_pop[i].notna()])
    avg = round(df_pop[i].mean(),2)
    median = round(df_pop[i].median(),2)
    max_val = df_pop[i].max()
    min_val = df_pop[i].min()
    myarr.append([effectif, avg, median, max_val, min_val])

test =  go.Table(header=dict(values=list(columns), align='left'), cells=dict(values= myarr,align='left'))

fig = go.Figure(data=test)
fig.update_layout(width=mywidth, height=myheight)
fig.show()

In [15]:
display(HTML('<h4>5. Tous les tests </h4>'))

df_pop = df[(df['TAP']=='Oui') &
            (df['Alerte phasique avec avertisseur : médiane - Temps (ms)'].notna()) &
            (df['Attention soutenue : médiane - Note brute'].notna()) &
            (df['MDT Condition 3 : médiane - Note brute'].notna()) &
            (df['Go / No go Condition 1 : médiane - Note brute'].notna())
           ]
total = len(df_pop)
age_avg = round(df_pop["age"].mean())
age_min = df_pop["age"].min()
age_max = df_pop["age"].max()

#Patients ayant une aide auditive
df_coaud = df_pop[(df_pop["Prothèse auditive DROITE"]!="Aucune") | (df_pop["Prothèse auditive GAUCHE"]!="Aucune") | (df_pop["Implant cochléaire DROIT"]!="Aucun") | (df_pop["Implant cochléaire GAUCHE"]!="Aucun")]
#Patients ayant uniquement une aide auditive
df_coaud_u = df_coaud[df_coaud["Correction visuelle"]=="Aucune"]
coaud_u = len(df_coaud_u)

#Patients ayant une correction visuelle
df_covis = df_pop[df_pop["Correction visuelle"]!="Aucune"]
#Patients ayant uniquement une correction visuelle
df_covis_u = df_covis[((df_covis["Prothèse auditive DROITE"]=="Aucune") & (df_covis["Prothèse auditive GAUCHE"]=="Aucune") & (df_covis["Implant cochléaire DROIT"]=="Aucun") & (df_covis["Implant cochléaire GAUCHE"]=="Aucun"))]
covis_u = len(df_covis_u)

#Patients ayant une aide auditive + visuelle
df_coaud_covis = df_coaud[df_coaud["Correction visuelle"]!="Aucune"]
coaud_covis = len(df_coaud_covis)

intlsf = len(df_pop[df_pop["Interprète LSF"]=="Oui"])

columns= ['','Valeur']
myarr = [["Effectif",
            'Moyenne d’âge',
            'Âge minimum',
            'Âge maximum',
            'Patients ayant uniquement une correction visuelle',
            'Patients ayant uniquement une correction auditive',
            'Patients ayant une correction auditive + une correction visuelle',
            'Nombre de personnes ayant recours un interprète']]
for i in columns[1:]:
    myarr.append([total, age_avg, age_min, age_max, covis_u, coaud_u, coaud_covis, intlsf])

str_data =  go.Table(
                columnwidth = [30,5],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=str_data)
fig.show()

###
display(HTML("<p>Répartition du nombre d'années d'études :</p>"))
ranges = [0,5,10,12,15,20]

df_study = df_pop["Nombre d'années d'études"].copy()

df_study = df_study.groupby(pd.cut(df_pop["Nombre d'années d'études"], ranges, include_lowest=True)).count()
df_study = pd.DataFrame(df_study)
df_study = df_study.rename(columns={"Nombre d'années d'études":'Effectif'})
year_study = ["0 à 5", "5 à 10", "10 à 12", "12 à 15", "15 à 20"]
df_study["Nombre d'années d'études"] = year_study
df_study = df_study[["Nombre d'années d'études", "Effectif"]]
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_study.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_study["Effectif"], df_study["Nombre d'années d'études"]],
               fill_color='lavender',
               align='left'))
])

fig.show()

fig = px.bar(df_study, x="Nombre d'années d'études", y="Effectif", color="Effectif")
fig.update_layout(showlegend=False)
fig.show()